In [1]:
from itertools import count

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score

# fix random seed for reproducibility
np.random.seed(1)

Using TensorFlow backend.


Load in the questions and associated tags for each question

In [2]:
q = pd.read_csv("../../stackoverflow-nlp/stacksample/Questions.csv", encoding='latin1')

In [3]:
t = pd.read_csv("../../stackoverflow-nlp/stacksample/Tags.csv", encoding='latin1')

In [4]:
titles = q.Title.str.split()

# Convert the words to a naive embedding
q_embeddings = set()

for title in titles:
    for word in title:
        q_embeddings.add(word.lower())

q_embeddings = dict(zip(q_embeddings, count()))
q.Title = q.Title.map(lambda x: [q_embeddings[word.lower()] for word in x.split()])

In [5]:
# Pad the titles to the same length
max_title_len = q.Title.apply(len).max()

q.Title = [np.concatenate([np.zeros(max_title_len - len(t)), t]) for t in q.Title]

Now we need to assign tags to each question.

In [6]:
# There's definitely a better way to do this but whatever
# This is really bad though

t = t.groupby('Id').first()

In [7]:
q = q.join(t, on='Id', rsuffix='t_')
q = q[~q.Tag.isna()]

In [8]:
# Split into training and testing data
train, test = train_test_split(q, train_size=0.7)

/home/mgray/Documents/Data_Science/Data_Munging_and_EDA/rybot/rybot_env/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
X_train = train.Title
X_test = test.Title

y_train = train.Tag
y_test = test.Tag

## LSTM Time

In [17]:
lb = LabelBinarizer()
lb.fit(q.Tag.unique())

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [11]:
# One-hot encoding runs out of memory if used on the entire training set
# so intead train by batches
# Could instead limit to top 1000 tags or so
X_train_batches = np.split(X_train, 39)
y_train_batches = np.split(y_train, 39)

In [12]:
# np_utils.to_categorical(y_train_batches[0])

def baseline_model(input_length=max_title_len, n_words=max(q_embeddings.values()),
                   embedding_vector_length=32, n_tags=9388):
    model = Sequential()
    model.add(Embedding(n_words, embedding_vector_length, input_length=input_length))
    model.add(Dense(10, input_dim=embedding_vector_length, activation='relu'))
    model.add(Flatten())
    model.add(Dense(n_tags, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [13]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [14]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
model.train_on_batch(np.array(X_train_batches[0].tolist()), 
                     np_utils.to_categorical(y_train_batches[0]))

In [ ]:
results = cross_val_score(estimator, np.array(X_train_batches[0].tolist()),
                          lb.transform(y_train_batches[0]), cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [131]:
X_train_batches[0]

66558      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
999375     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
874475     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
894880     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
33364      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
840798     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
617415     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
346607     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
573952     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
386158     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
185658     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
18584      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
845381     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
737672     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1224058    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
428185     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
449817     [0.0, 0.0, 0.